# Theoretical Analysis
> Theoretical Analysis (level 0) for all CNN topologies and hardware platforms

- toc: true 
- badges: true
- comments: true
- categories: [Rooflines,MNIST,ImageNet,CIFAR-10]
- image: images/roofline.png

In [1]:
#hide
import numpy as np
import pandas as pd
import random

pd.options.display.max_rows = 10000 # this will set limit of columns to 500
pd.options.display.float_format = '{:20,.0f}'.format
import altair as alt
W = 600
H = 480

In [2]:
#hide
#notes
#rn50_df = df[df.apply(lambda row: row['net_prun_datatype'].split('_')[0] == 'MobileNetv1|ResNet50', axis=1)]
#rn50_df['net_prun_datatype'] = rn50_df['net_prun_datatype'].str.split('_').str[0] #comment this when I want to hide the topology
#rn50_df['hardw'] = rn50_df['hardw'].str.replace("_INT8|_FP16|_FP32", "")

# Introduction

This page presents a Theoretical Analysis of both hardware platforms as well as CNN topologies.
In order to get a general overview of all CNNs and Hardware Platforms included in our experiments, we present the following 3 tables. 

# Tables

### CNNs and Their Accuracy Over All Pruning and Quantization Variants

Table below provides a complete overview of all CNNs that were included in the experimentation and their corresponding accuracy over all Pruning and Quantization Variants.

In [3]:
#hide_input
%run scripts/script_tables.py  #run the heatmaps script
tableOverviewExperiments(['data/cnn_topologies_accuracy.csv'])

,INT2,INT4,INT8,FP16,FP32
,top1 (top5) [%],top1 (top5) [%],top1 (top5) [%],top1 (top5) [%],top1 (top5) [%]
GoogLeNetv1,nm,nm,69.24 (88.45),66.93 (87.83),66.96 (87.84)
MobileNetv1,nm,nm,69.57 (87.71),nm,nm
EfficientNet-S 100%,nm,nm,77,nm,nm
EfficientNet-M 100%,nm,nm,78.6,nm,nm
EfficientNet-L 100%,nm,nm,80.2,nm,nm
ResNet50 100%,nm,nm,73.29 (91.26),75.14 (92.12),75.15 (92.11)
ResNet50 80%,nm,nm,73.30 (91.40),nm,nm
ResNet50 50%,nm,nm,69.49 (91.00),nm,nm
ResNet50 30%,nm,nm,68.83 ( 90.16),nm,nm


#hide
# -----------------Overlapped paretos------------------

## Some needed functions

In [37]:
#hide
def melt_df(df_in: pd.DataFrame(), cnn_names_col: str, new_column_names: list)->pd.DataFrame():
    """Melts a dataframe into 2 columns, the 'cnn_names_col' and the 'value' column. Return the melted dataframe"""
    df_out = pd.DataFrame()
    #  select all columns except first
    columns = (df_in.loc[:, df_in.columns!=cnn_names_col]).columns 
    for column in columns:
        # melt df1 to have only 2 columns
        df_tmp = pd.melt(df_in, id_vars=[cnn_names_col], value_vars=column) 
        df_out = pd.concat([df_out,df_tmp])
    # setting new columns names
    df_out.columns = new_column_names 
    return df_out

In [38]:
#hide
def spot_no_match(list_: list):
    """Creates a list of hexadecimal colors. The colors depend wheteher there is a substring inside each
    list item. For 'no match' the color is black, else, the color is created randomly
    
    """
    sub='no_match'
    list_of_colors=[]
    for index, word in enumerate(list_):
        #if there is no match then appned the black color
        if sub in word:
            list_of_colors.append('#000000')
        else:
            # create random color
            color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])]
            list_of_colors.append(color[0])
    return list_of_colors
        

In [39]:
#hide
def get_point_chart_simple(df: pd.DataFrame, 
                    color_groupcol: str, 
                    shape_groupcol: str,  
                    xcol: str, 
                    ycol: str, 
                    shapes: str,
                    title: str,
                    legend_title_groupcol: str)->alt.vegalite.v4.api.Chart: 
    """Creates an elaborated point chart with all these configurations"""
    domain = df[color_groupcol].unique().tolist()
    range_= spot_no_match(list_= domain)
    points= alt.Chart(df).mark_point(size=100, opacity=1, filled =True).properties(
            width= W,
            height= 1.3*H,
            title=title
        ).encode(
            x= alt.X(xcol,  scale=alt.Scale(type="log")),
            y=alt.Y(ycol + ":Q", scale=alt.Scale(zero=False)),
            color=alt.Color(color_groupcol, scale=alt.Scale(domain=domain, range=range_), legend=alt.Legend(columns=2, title = legend_title_groupcol)),
            #tooltip=["HWType", "Precision", "PruningFactor", "batch/thread/stream", ycol, xcol],
            shape=alt.Shape(shape_groupcol, scale=alt.Scale(range=shapes)),
            tooltip=['hardw_datatype_net_prun',color_groupcol, shape_groupcol, xcol, ycol],

        )
    text = points.mark_text(
        angle=325,
        align='left',
        baseline='middle',
        dx=7
    ).encode(
        text='hardw'
    )
    return (points + text).interactive()

In [40]:
#hide
def get_line_chart(df: pd.DataFrame, groupcol: str, xcol: str, ycol:str, color:str) ->alt.vegalite.v4.api.Chart:
    """Creates simple line chart"""
    return alt.Chart(df).interactive().mark_line(point=True).encode(
        x=xcol,
        y=alt.Y(ycol + ":Q", scale=alt.Scale(zero=False)),
        color=alt.value(color),
        tooltip=[groupcol, xcol, ycol],
    )

In [41]:
#hide
def get_pareto_df(df: pd.DataFrame(), groupcol: str, xcol: str, ycol: str) -> pd.DataFrame():
    """Creates a pareto line from the dataframe. This function doesn't correctly correspond x to y datapoints"""
    pareto_line_df = df.groupby(groupcol)[xcol].max().to_frame("x")
    pareto_line_df['y'] = df.groupby(groupcol)[ycol].agg(lambda x: x.value_counts().index[0])
    pareto_line_df.sort_values('y', ascending=False, inplace=True)
    pareto_line_df['x'] = pareto_line_df.x.cummax()
    pareto_line_df.drop_duplicates('x', keep='first', inplace=True)
    pareto_line_df['group'] = pareto_line_df.index
    return pareto_line_df

In [42]:
#hide
def get_several_paretos_df(list_df: pd.DataFrame, groupcol: str, xcol: str, ycol:str, colors: list)->list:
    """Method that: from the input dfs creates a pareto df & creates a plot from the pareto df, does this for all input dfs"""
    list_df_out_charts = pd.DataFrame(columns=['charts'])
    for i, df in enumerate(list_df):
        pareto_df = get_pareto_df(df= df , groupcol= groupcol, xcol= xcol, ycol= ycol)
        chart = get_line_chart(df= pareto_df, groupcol= 'group', xcol= 'x', ycol= 'y', color = colors[i]) 
        list_df_out_charts = list_df_out_charts.append(pd.DataFrame([[chart]], columns=['charts']))
    return list_df_out_charts

In [43]:
#hide
def replace_data_df(df_: pd.DataFrame(), column:str, list_tuples_data_to_replace: list )-> pd.DataFrame():
    """Method to replace a substring inside a cell inside a dataframe"""
    df = df_.copy()
    for j, k in list_tuples_data_to_replace:
        df[column] = df[column].str.replace(j, k)
    return df

In [44]:
#hide
def process_measured_df(df_theoret: pd.DataFrame(), csv_measured:str ):
    #   get the measured dataframe
    df_measured = pd.read_csv(csv_measured)
    #   fix samll stuff in the measured dataframe so things match
    df_measured = replace_data_df(df_=df_measured, column='hardw_datatype_net_prun', list_tuples_data_to_replace=[("RN50", "ResNet50"),("MNv1", "MobileNetv1"),('GNv1','GoogLeNetv1'),('100.0','100')])
    df_measured = replace_data_df(df_=df_measured, column='network', list_tuples_data_to_replace=[("RN50", "ResNet50"),("MNv1", "MobileNetv1"),('GNv1','GoogLeNetv1')])
    #  concatenate both measured with theoretical
    df = pd.concat([df_theoret, df_measured])

    

#hide
### Lets get to the processing

In [45]:
#hide
def select_cnn_match_theo_for_measured(df_superset: pd.DataFrame(),
                     cnn_keyword: str,
                     net_prun_datatype: str,
                     ):
    """This method creates an overlepped pareto"""
    #   create a subset from the given dataframe
    # there is another way to do this # df_theo = df_superset[df_superset.apply(lambda row: row[net_prun_datatype].split('_')[0] == cnn_keyword, axis=1)]
    df_theo = df_superset.loc[df_superset[net_prun_datatype].str.contains(cnn_keyword, na=False)]
    df_theo = df_theo[df_theo['top1'].notna()]
    df_theo = df_theo[df_theo['fps-comp'].notna()]
    #   given that we have on theoretical df:  net_prun_datatype | hardw_datatype | top1 | fps-comp
    #   and that we have on the measured df:   hardw_datatype_net_prun | batch/thread/stream  | hardw | network | fps-comp | top1 | type
    #We need to:
    #   1. Create 'network', 'type', 'hardware' and 'hardw_datatype_net_prun'
    df_theo['network'] = df_theo['net_prun_datatype'].str.split('_').str[0]
    df_theo['type'] = 'theoretical'
    #replace elemnts out of hardw column - take datatypes out of hardw_datatype column
    df_theo = replace_data_df(df_=df_theo, column= 'hardw_datatype', list_tuples_data_to_replace=[("-INT2", ""), ("-INT4", ""), ("-INT8", ""), ("-FP16", ""), ("-FP32", "")])      
    # 'hardw_datatype' column only has the hardware now
    df_theo['hardw_datatype_net_prun'] = df_theo['hardw_datatype']+'_'+df_theo['net_prun_datatype'].str.split('_').str[2] +'_'+ df_theo['network']+'_'+df_theo['net_prun_datatype'].str.split('_').str[1]
        
    #   delet unnecessary columns
    df_theo = df_theo.drop(columns = ['net_prun_datatype'])
    #  change column order
    df_theo= df_theo[['hardw_datatype_net_prun', 'hardw_datatype','network', 'fps-comp', 'top1', 'type']]

    #   rename columns
    df_theo.columns=['hardw_datatype_net_prun','hardw','network', 'fps-comp', 'top1', 'type']

    
    return df_theo

In [46]:
#hide
def fix_small_stuff_df(df: pd.DataFrame(), col_to_drop: list, ) -> pd.DataFrame():
    #   delete all rows that have 'top1 (top5) [%]' inside
    df = df[df['top1'] !='top1 (top5) [%]']
    #    delete all rows with 'nm'
    df = df[df.top1!='nm'] 
    df = df.reset_index()
    #   merge 'net_prun' with 'datatype' column into 'net_prun_datatype'
    df['net_prun_datatype'] = df.net_prun + ' ' + df.datatype
    df = df.drop(columns = col_to_drop)

    #    Some cells have [top1 (top5)] accuracies, create col only with top1 acc
    df['top1'] = df['top1'].str.split(' ').str[0] #take top5 acc out
    #    separate by underscore instead of space
    df = replace_data_df(df_=df, column='net_prun_datatype', list_tuples_data_to_replace=[(' ','_')])
    return df

In [47]:
#hide
def process_theo_top1(csv_theor_accuracies: str) -> pd.DataFrame():
    # GET THEORETICAL values
    #  get the table above
    df_top1_theo = pd.read_csv(csv_theor_accuracies)
    #  melt it into 2 columns: 
    df_top1_theo = melt_df(df_in= df_top1_theo, cnn_names_col= ' ', new_column_names=['net_prun','datatype','top1'])
    #fix small stuff like deleting rows, merging columns...
    df_top1_theo = fix_small_stuff_df(df= df_top1_theo, col_to_drop=['index','datatype','net_prun'] )
    #  now we have: top1 | net_prun_datatype 
    return df_top1_theo

In [48]:
#hide
def process_theo_fps(df_top1_theo:pd.DataFrame(),csv_files: list) -> pd.DataFrame():
    #    GET Heatmaps/Theoretical TABLES
    # get fps data from csv from heatmaps and mrge them on the 'net_prun_datatype' common column.
    # Notes: 1. values need to be the sam. 
    #        Eg.: 'MLP_100%_INT2' has to match with 'MLP_100%_INT2' otherwise what comes from the performance precitions will be ignored
    df_fps_theo = pd.DataFrame()
    for csv_file in csv_files:
        df_tmp = pd.read_csv(csv_file)
        df_fps_theo = pd.concat([df_fps_theo, df_tmp])

    df_fps_theo['x']= df_fps_theo['x'].str.replace('-','_')
    #    remove rows that have 'nan' in the 'values' column
    df_fps_theo = df_fps_theo[df_fps_theo['values'].notna()]
    #    rename columns
    df_fps_theo.columns=['hardw','net_prun_datatype','fps']

    #   Merge both Theoretical dataframes: fps + top1 
    df_fps_top1_theo = pd.merge(df_top1_theo, df_fps_theo, on='net_prun_datatype', how='outer')
    #  change column order
    df_fps_top1_theo = df_fps_top1_theo[['net_prun_datatype', 'hardw', 'top1', 'fps']]
    #  change column names
    df_fps_top1_theo.columns = ['net_prun_datatype', 'hardw_datatype', 'top1', 'fps-comp']

    #Notes: 1. make sure everything in 'net_prun_datatype' column has network + prunning + datatype. If not it will fail
    df_fps_top1_theo = replace_data_df(df_=df_fps_top1_theo, column= 'net_prun_datatype', list_tuples_data_to_replace= [('GoogLeNetv1','GoogLeNetv1_100%'),('MobileNetv1','MobileNetv1_100%'),('GoogleNetv1','GoogleNetv1_100%'), ('EfficientNet_S','EfficientNet-S_100%'), ('EfficientNet_M','EfficientNet-M_100%'), ('EfficientNet_L','EfficientNet-L_100%'), ('%','')])
    #  now that we have: net_prun_datatype | hardw_datatype | top1 | fps-comp
    return df_fps_top1_theo

In [49]:
#hide
def identify_pairs_nonpairs(df: pd.DataFrame, column: str) -> pd.DataFrame():
    """This method identifies equal values in the column and signals them, and creates another column with labels for each case """
    # IDENTIFY ALL PAIRS AND CREATE A SPECIAL COLUMN FOR THEM
    #get all pair and then get unique names out of those pairs
    df['pairs'] = df[column].duplicated(keep=False)
    unique_names = df.loc[df.pairs ==True, column].unique()
    #set a color for each one of them
    color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             for i in range(len(unique_names))]
    #put it into a dict
    names_with_colors = {key:color for key,color in zip(unique_names,color)}
    #assign it to the dataframe color column. Only fill up rows (with the same color) that have pairs
    df['color'] = df[column].apply(lambda x: x if x in names_with_colors else '')
    #fill up the rest of the rows that do not have a pair
    df['color'] = df.apply(lambda row: 'theoretical_no_match' if row.type=='theoretical' and row.color=='' else 
                                                     ('experimental_no_match' if row.type=='experimental' and row.color=='' else (row.color)), axis=1)
    df = df.drop(columns=['pairs'])
    return df

In [53]:
#hide
def plot_it_now(df: pd.DataFrame, xcol: str, ycol: str, groupcol: str, title: str):
    """This method gets makes the overlapped pareto plots. With the 2 pareto lines and the points plot"""
    df_theo =df.loc[df.type=='theoretical',:]
    df_exper = df.loc[df.type=='experimental',:]
    df_charts = get_several_paretos_df(list_df = [df_theo, df_exper], groupcol= groupcol, xcol= xcol , ycol= ycol, colors=['#FFA500', '#0066CC'])
    chart1 = get_point_chart_simple(df= df, color_groupcol= 'color', shape_groupcol= 'type',shapes=['cross', 'circle'], xcol= xcol, ycol= ycol, title=title, legend_title_groupcol="Hardw_Datatype_Net_Prun" )
    chart = df_charts.charts.sum(numeric_only = False)
    charts = alt.layer(
        chart1,
        chart
    ).resolve_scale(color='independent',shape='independent').properties(title=title)
    return charts

#hide
### Get all different topologies first

In [54]:
#hide
# Plan:  1. Get top1 -> Get fps -> Get measured pareto -> join them -> identify pairs -> plot it

# 1. Get the CNNs Accuracies table ( from above) that only has the top1 accuracy and process it.
#   theoretical top1
df_top1_theo = process_theo_top1(csv_theor_accuracies ='data/cnn_topologies_accuracy.csv')
#now we have: |top1 | net_prun_datatype|

# 2. Now we need Theoretical FPS-COMP to match with that Theoretical TOP1
# 3. We need to get the above mentioned Theoretical FCPS-COMP from the Heatmaps- Performance Predictions and merge them
#   theoretical top1 + fps
df_fps_top1_theo = process_theo_fps(df_top1_theo= df_top1_theo, csv_files=["data/cleaned_csv/performance_prediction_cifar10.csv",
                                               "data/cleaned_csv/performance_prediction_imagenet.csv",
                                               "data/cleaned_csv/performance_prediction_mnist.csv"])
#now we have: |net_prun_datatype | hardw_datatype | top1 | fps-comp|


#divide by classification and process this theoretical df to match to experimental df
#df_fps_top1_theo.net_prun_datatype.str.split('_').str[0].unique() #this is to get all unique networks to fill up th ecnn_keyword down here
df_theor_mnist    = select_cnn_match_theo_for_measured(df_superset= df_fps_top1_theo, cnn_keyword= 'MLP', net_prun_datatype = 'net_prun_datatype')
df_theor_cifar_10 = select_cnn_match_theo_for_measured(df_superset= df_fps_top1_theo, cnn_keyword= 'CNV', net_prun_datatype = 'net_prun_datatype')
df_theor_imagenet = select_cnn_match_theo_for_measured(df_superset= df_fps_top1_theo, cnn_keyword= 'ResNet50|MobileNetv1|EfficientNet|GoogleNetv1', net_prun_datatype = 'net_prun_datatype')
# now we have: |hardw_datatype_net_prun | hardw | network | fps-comp | top1 | type|

#   measured pareto df
df_measured_imagenet = pd.read_csv('data/cleaned_csv/pareto_data_imagenet.csv')
df_measured_mnist    = pd.read_csv('data/cleaned_csv/pareto_data_mnist.csv')
df_measured_cifar10  = pd.read_csv('data/cleaned_csv/pareto_data_cifar.csv')

#  concatenate both measured with theoretical to get the overlapped pareto
overlapped_pareto_imagenet = pd.concat([df_theor_imagenet, df_measured_imagenet])
overlapped_pareto_mnist    = pd.concat([df_theor_mnist,    df_measured_mnist])
overlapped_pareto_cifar10  = pd.concat([df_theor_cifar_10, df_measured_cifar10])
# now we have everything together and matched

overlapped_pareto_imagenet.sort_values(by='hardw_datatype_net_prun')
overlapped_pareto_mnist.sort_values(by='hardw_datatype_net_prun')
overlapped_pareto_cifar10.sort_values(by='hardw_datatype_net_prun')


# identify all pairs and create a special column for them 
overlapped_pareto_imagenet = identify_pairs_nonpairs(df=overlapped_pareto_imagenet, column='hardw_datatype_net_prun')
overlapped_pareto_mnist    = identify_pairs_nonpairs(df=overlapped_pareto_mnist, column='hardw_datatype_net_prun')
overlapped_pareto_cifar10  = identify_pairs_nonpairs(df=overlapped_pareto_cifar10, column='hardw_datatype_net_prun')
# now we have: |hardw_datatype_net_prun | hardw | network | fps-comp | top1 | type | color|

In [56]:
#hide_input
plot_it_now(df= overlapped_pareto_imagenet, xcol= 'fps-comp', ycol= 'top1', groupcol= 'hardw_datatype_net_prun', title='Overlapped Pareto Plots Theoretical + Measured for Imagenet')

alt.LayerChart(...)

In [58]:
#hide_input
plot_it_now(df= overlapped_pareto_cifar10, xcol= 'fps-comp', ycol= 'top1', groupcol= 'hardw_datatype_net_prun', title='Overlapped Pareto Plots Theoretical + Measured for CIFAR-10')

alt.LayerChart(...)

In [60]:
#hide_input
plot_it_now(df= overlapped_pareto_mnist, xcol= 'fps-comp', ycol= 'top1', groupcol= 'hardw_datatype_net_prun', title='Overlapped Pareto Plots Theoretical + Measured for MNIST')

alt.LayerChart(...)

### CNNs and Their Compute and Memory Requirements

Next table shows Compute and Memory Requirements for all CNNs in number of operations ([GOPs]), Model Size ([ME]) and Operational Intensity ([OI]) in operations per byte read or written from memory.

In [26]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
tableOverviewExperiments(['data/cnn_topologies_compute_memory_requirements.csv'])

,Total OPs,Total Model Size,OI (INT2),OI (INT4),OI (INT8),OI (FP16),OI (FP32)
,GOPs,[ME],[Ops/Byte],[Ops/Byte],[Ops/Byte],[Ops/Byte],[Ops/Byte]
GoogLeNetv1,3.1,6,2093.97,1046.99,523.49,261.75,130.87
MobileNetv1,1.1,4.2,1075.47,537.74,268.87,134.43,67.22
ResNet-50 100%,7.7,25.5,1210.84,605.42,302.71,151.36,75.68
ResNet-50 80%,6.5,23.7,1086.59,543.3,271.65,135.82,67.91
ResNet-50 50%,3.8,15.8,949.85,474.93,237.46,118.73,59.37
ResNet-50 30%,2.5,10.1,970.16,485.08,242.54,121.27,60.64
EfficientNet Edge L,4.7,5.4,3481.48,1740.74,870.37,435.18,217.59
EfficientNet Edge M,7.4,6.9,4289.86,2144.93,1072.46,536.23,268.12
EfficientNet Edge S,19.4,10.6,7313.21,3656.6,1828.3,914.15,457.08


## Hardware Platforms

Table below summarizes all included hardware platforms, each with its corresponding peak performance for diferent datatypes (INTx, FPx), its Memory Bandwidth, Memory capacity as well as Thermal Design Power.

In [27]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
tableOverviewExperiments(['data/hardware_platforms.csv'])

Hardware Platforms,INT2,INT4,INT8,FP16,FP32,Memory Bandwidth,Memory Capacity,Power
,[TOP/sec],[TOP/sec],[TOP/sec],[TOP/sec],[TOP/sec],[GBps],[GB],[Watt]
Ultra96-DPU,na,na,0.96,na,na,4.26,2,na
ZCU104-DPU,na,na,4.6,na,na,19.2,4,na
ZCU102-DPU,na,na,6.71,na,na,19.2,4,na
ZCU104-FINN,30.7,8.8,na,na,na,19.2,4,na
ZCU104-BISMO,30.7,8.8,na,na,na,19.2,4,na
TX2 - maxn,na,na,na,1.33,0.67,59.7,8,15
TX2 - maxp,na,na,na,1.15,0.57,59.7,8,15
TX2 - maxq,na,na,na,0.87,0.44,59.7,8,15
TPU-fast,na,na,4,na,na,25.6,1,2


## Overview of Theoretical Evaluation

link to: https://rcl-lab.github.io/Qutibench_Web/mnist/imagenet/cifar-10/2020/04/30/Overview_of_experiments.html

# Rooflines for all Hardware Platforms and CNNs

Combining application requirements with hardware platform characteristics can be leveraged for performance predictions using UCB’s roofline models. Using assumptions for where weights, activation tensors, and state of a neural network are stored, combined with the size of the datatypes used, allow us to derive the arithmetic intensity of a neural network during inference. Combined with the roofline for a given hardware platform, we can provide insight as to whether a neural network will be memory or compute bound and guidance for what is theoretically possible in regards to its throughput.

In [28]:
#hide_input

#first process the following csv's to get clean ready to plot csv's
%run scripts/script_load_save_data.py
clean_csv_rooflines(path_topologies='c:/Users/alinav/Documents/GitHub/Qutibench_Web/_notebooks/data/topology_details.csv',
                    path_hardware='c:/Users/alinav/Documents/GitHub/Qutibench_Web/_notebooks/data/peakPerfBandHardPlatf.csv')

#Now get the cleaned csv, and plot it as a Roofline
%run scripts/altair_plots.py
rooflines(pd.read_csv("data/cleaned_csv/rooflines_hardware_neural_networks.csv"), 'imagenet|mnist|cifar')

alt.LayerChart(...)

# Performance Prediction

The following heatmaps show the theoretical performance for the listed hardware platforms across the various machine learning tasks: MNIST, ImageNet and CIFAR-10. The metric used for the theoretical performance is input/second.

In [29]:
#hide
# First process the unfiltered csv and save it as a pretty csv ready to plotted as a heatmap
#%run scripts/script_load_save_data.py
#have to change clean_csv_performance_predictions to process the right csv the right way
#clean_csv_performance_predictions('c:/Users/alinav/Documents/GitHub/QutibenchWeb/_notebooks/data/performance_predictions_imagenet_mnist_cifar.csv')

### MNIST

For MNIST, quantization combined with pruning deliver some of best performance results.

In [30]:
#hide_input
%run scripts/altair_plots.py
#load mnist dataset and plot it
heatmap(pd.read_csv("data/cleaned_csv/performance_prediction_mnist.csv"), 'red', 'Performance Prediction for MNIST')

alt.LayerChart(...)

### ImageNet

For ImageNet, quantization combined with pruning also deliver some of best performance results.

In [31]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
#load imagenet dataset and plot it
heatmap(pd.read_csv("data/cleaned_csv/performance_prediction_imagenet.csv"), 'lightgrey','Performance Prediction for Imagenet')

alt.LayerChart(...)

### CIFAR-10

Finally, for CIFAR-10, quantization combined with pruning deliver some of best performance results

In [32]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
#load cifar10 dataset and plot it
heatmap(pd.read_csv("data/cleaned_csv/performance_prediction_cifar10.csv"), 'pink','Performance Prediction for CIFAR-10')

alt.LayerChart(...)